In [1]:
from stravalib import Client
import jupyter_black
from datetime import datetime, timedelta
import pandas
from stravalib import Client
import altair as alt
import numpy

jupyter_black.load()

MY_STRAVA_CLIENT_ID = 125463
MY_STRAVA_CLIENT_SECRET = "714fac0667a26729064973b0d01b9e317020a956"

In [2]:
def authorize() -> Client:
    client = Client()
    url = client.authorization_url(
        client_id=125463, redirect_uri="http://127.0.0.1:5000/authorization"
    )

    print(f"Go to {url}")
    response = input(f"Enter the full URL:")
    code = response.split("code=")[1].split("&")[0]
    token_response = client.exchange_code_for_token(
        client_id=MY_STRAVA_CLIENT_ID, client_secret=MY_STRAVA_CLIENT_SECRET, code=code
    )
    access_token = token_response["access_token"]
    refresh_token = token_response["refresh_token"]

    return Client(access_token=access_token)


# client = Client()
# token_response = client.refresh_access_token(client_id=MY_STRAVA_CLIENT_ID,
#                                       client_secret=MY_STRAVA_CLIENT_SECRET,
#                                       refresh_token=last_refresh_token)
# new_access_token = token_response['access_token']

client = authorize()

Go to https://www.strava.com/oauth/authorize?client_id=125463&redirect_uri=http%3A%2F%2F127.0.0.1%3A5000%2Fauthorization&approval_prompt=auto&scope=read%2Cactivity%3Aread&response_type=code


In [3]:
five_months_ago = datetime.now() - timedelta(days=180)
activities = client.get_activities(after=five_months_ago.isoformat())
running_activities = [act for act in activities if act.type == "Run"]

In [10]:
data = {
    "start_date": [act.start_date for act in running_activities],
    "distance_meters": [float(act.distance) for act in running_activities],
    "time_seconds": [act.elapsed_time.total_seconds() for act in running_activities],
}


df = pandas.DataFrame(data)
df["start_date"] = pandas.to_datetime(df["start_date"])
df.set_index("start_date", inplace=True)
weekly_data = df.resample("W").sum()
weekly_data["distance_km"] = round(weekly_data["distance_meters"] / 1000, 1)
upper_limit = weekly_data["distance_km"].mean() + 2 * weekly_data["distance_km"].std()

chart = (
    alt.Chart(weekly_data.reset_index())
    .mark_bar(size=30)
    .encode(
        x="start_date:T",
        y=alt.Y("distance_km:Q", scale=alt.Scale(domain=[0, upper_limit])),
        tooltip=["start_date", "distance_km"],
    )
)


text_labels = (
    alt.Chart(weekly_data.reset_index())
    .mark_text(dy=-10, color="black")
    .encode(
        x="start_date:T",
        y=alt.Y("distance_km:Q", scale=alt.Scale(domain=[0, upper_limit])),
        text="distance_km:Q",
    )
)

(chart + text_labels).properties(
    width=800, height=500, title="Running distance per week (km)"
).interactive().show()

alt.LayerChart(...)

In [11]:
def get_tanda_value(km_per_week: int, pace_sec_per_km: int) -> float:
    marathon_distance = 42.195
    marathon_pace_sec_per_km = (
        17.1 + 140.0 * numpy.exp(-0.0053 * km_per_week) + 0.55 * pace_sec_per_km
    )
    total_marathon_time_secs = marathon_distance * marathon_pace_sec_per_km
    total_marathon_time_hours = total_marathon_time_secs / 3600
    return total_marathon_time_hours


def pretty_marathon_time(total_marathon_time_hours: float) -> str:
    hours = int(total_marathon_time_hours)
    minutes = int((total_marathon_time_hours - hours) * 60)
    seconds = int(((total_marathon_time_hours - hours) * 60 - minutes) * 60)
    if seconds >= 30:
        minutes += 1

    return f"{hours} hours {minutes} minutes"

In [12]:
daily_df = df.groupby(df.index.date).sum()
daily_df.index = pandas.to_datetime(daily_df.index)
daily_df.index.name = "date"

In [13]:
daily_df["tanda_day"] = get_tanda_value(
    daily_df["distance_meters"] / 1000 * 7,
    daily_df["time_seconds"] / (daily_df["distance_meters"] / 1000),
)
daily_df["tanda_day_pretty"] = pandas.to_datetime(daily_df["tanda_day"], unit="h")


daily_df = daily_df.reset_index()
daily_df["date"] = pandas.to_datetime(daily_df["date"])
daily_df.set_index("date", inplace=True)

num_weeks = 8
num_days = num_weeks * 7
rolling = f"{num_days}d"


daily_df["rolling_distance_meters"] = (
    daily_df["distance_meters"].rolling(window=rolling).sum()
)
daily_df["rolling_time_seconds"] = (
    daily_df["time_seconds"].rolling(window=rolling).sum()
)

daily_df["rolling_km_per_week"] = daily_df["rolling_distance_meters"] / 1000 / num_weeks
daily_df["rolling_pace_sec_per_km"] = (
    daily_df["rolling_time_seconds"] / daily_df["rolling_distance_meters"] * 1000
)

daily_df["rolling_tanda_day"] = get_tanda_value(
    daily_df["rolling_km_per_week"], daily_df["rolling_pace_sec_per_km"]
)

daily_df["rolling_tanda_day_pretty"] = pandas.to_datetime(
    daily_df["rolling_tanda_day"], unit="h"
)

daily_df["type_rolling"] = "Tanda (8 weeks)"
daily_df["type_daily"] = "Tanda (daily)"


daily_df

,distance_meters,time_seconds,tanda_day,tanda_day_pretty,rolling_distance_meters,rolling_time_seconds,rolling_km_per_week,rolling_pace_sec_per_km,rolling_tanda_day,rolling_tanda_day_pretty,type_rolling,type_daily
date,,,,,,,,,,,,
2023-10-28,5000.0,1808.0,3.894557,1970-01-01 03:53:40.404258427,5000.0,1808.0,0.625000,361.600000,4.166956,1970-01-01 04:10:01.040542381,Tanda (8 weeks),Tanda (daily)
2023-10-29,5000.8,1811.0,3.898011,1970-01-01 03:53:52.838275624,10000.8,3619.0,1.250100,361.871050,4.163294,1970-01-01 04:09:47.856811485,Tanda (8 weeks),Tanda (daily)
2023-10-31,4660.4,2035.0,4.395696,1970-01-01 04:23:44.506141663,14661.2,5654.0,1.832650,385.643740,4.311518,1970-01-01 04:18:41.465069884,Tanda (8 weeks),Tanda (daily)
2023-11-02,5050.0,1810.0,3.871504,1970-01-01 03:52:17.414412855,19711.2,7464.0,2.463900,378.667965,4.261121,1970-01-01 04:15:40.036656456,Tanda (8 weeks),Tanda (daily)
2023-11-05,9280.0,3181.0,3.573103,1970-01-01 03:34:23.169584092,28991.2,10645.0,3.623900,367.180386,4.177140,1970-01-01 04:10:37.704593769,Tanda (8 weeks),Tanda (daily)
...,...,...,...,...,...,...,...,...,...,...,...,...
2024-04-13,8040.8,2637.0,3.532231,1970-01-01 03:31:56.030256440,384733.0,127404.0,48.091625,331.149135,3.606884,1970-01-01 03:36:24.783139742,Tanda (8 weeks),Tanda (daily)
2024-04-14,19581.9,6194.0,3.033076,1970-01-01 03:01:59.073118312,404314.9,133598.0,50.539363,330.430563,3.585860,1970-01-01 03:35:09.097787068,Tanda (8 weeks),Tanda (daily)
2024-04-16,7018.6,2229.0,3.512461,1970-01-01 03:30:44.858831450,403307.8,133137.0,50.413475,330.112634,3.584649,1970-01-01 03:35:04.735740559,Tanda (8 weeks),Tanda (daily)


In [14]:
daily_line = (
    alt.Chart(daily_df.reset_index())
    .mark_point()
    .encode(
        x=alt.X("date:T", title="Date"),
        y=alt.Y("hoursminutes(tanda_day_pretty):O", title="Tanda day"),
        color=alt.Color(
            "type_daily:N",
            legend=alt.Legend(title="Legend"),
        ),
        tooltip=[
            alt.Tooltip("tanda_day_pretty", timeUnit="hoursminutes"),
            alt.Tooltip("date", timeUnit="yearmonthdate"),
        ],
    )
)
rolling_line = (
    alt.Chart(daily_df.reset_index())
    .mark_line()
    .encode(
        x=alt.X("date:T", title="Date"),
        y=alt.Y("hoursminutes(rolling_tanda_day_pretty):O", title="Tanda (8 weeks)"),
        color=alt.Color(
            "type_rolling:N",
            legend=alt.Legend(title="Legend"),
        ),
        tooltip=[
            alt.Tooltip("rolling_tanda_day_pretty", timeUnit="hoursminutes"),
            alt.Tooltip("date", timeUnit="yearmonthdate"),
        ],
    )
)


(daily_line + rolling_line).properties(
    width=800, height=500, title="Tanda"
).interactive().show()

alt.LayerChart(...)